<H1> This is the week 3 assignment. 

<H3> resolving the environment with all necessary imports 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import geocoder as gc 
import folium as fl
from bs4 import BeautifulSoup as bs
import xml
import matplotlib as mlb
import json
import requests

<H4> Importing of all packages done!, now arranging url for data extraction from wikipedia 

In [ ]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(url,'lxml')

<H4> Identify the table and extract the data  as mentioned in instructions according to Borough, neighbourhood

In [ ]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_PostCode = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_PostCode.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostCode.head()

<H4> Remove Not Assigned values from dataframe and replace them by the aggregate values

In [ ]:
df_PostCode.head(7)

In [ ]:
df_PostCode['Borough'].replace('Not assigned', np.nan, inplace=True)
df_PostCode.dropna(subset=['Borough'], inplace=True)

df_PostCode.head(7)

In [ ]:
df_PostCodeNeigh = df_PostCode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_PostCodeNeigh.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostCodeNeigh

In [ ]:
df_PostCodeNeigh['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
df_PostCodeNeigh

In [ ]:
df_PostCodeNeigh.shape

In [ ]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [ ]:
df_pos = pd.merge(df_PostCodeNeigh, df_geo, on=['Postcode'], how='inner')

df_tor = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_tor.head()